In [31]:
import requests
import xml.etree.ElementTree as ET

API_KEY = "561433A49188D3CB67FEDC2120EC2C87"
query = "가을"

# 1단계: search API로 target_code 얻기
search_url = "https://opendict.korean.go.kr/api/search"
search_params = {
    'key': API_KEY,
    'q': query,
    'req_type': 'xml',
    'num': 10,
}

response = requests.get(search_url, params=search_params)
root = ET.fromstring(response.content)

item = root.find('.//item')
if item is None:
    print("검색 결과가 없습니다.")
else:
    word = item.findtext('word')
    target_code = item.find('sense').findtext('target_code')
    
    # 2단계: view API로 상세 정보 조회
    view_url = "https://opendict.korean.go.kr/api/view"
    view_params = {
        'key': API_KEY,
        'method': 'target_code',
        'q': target_code,
        'req_type': 'xml',
    }
    
    view_response = requests.get(view_url, params=view_params)
    view_root = ET.fromstring(view_response.content)
    
    # === WORD INFO 상세 정보 ===
    word_info = view_root.find('.//wordInfo')
    sense_info = view_root.find('.//senseInfo')
    
    print("=" * 50)
    print("📖 단어 정보 (WORD INFO)")
    print("=" * 50)
    
    # 기본 단어 정보
    word = word_info.findtext('word')
    word_unit = word_info.findtext('word_unit')
    word_type = word_info.findtext('word_type')
    origin = word_info.findtext('origin')
    allomorph = word_info.findtext('allomorph')
    
    print(f"🔹 표제어: {word}")
    print(f"🔹 구성 단위: {word_unit}")
    print(f"🔹 고유어 여부: {word_type}")
    if origin:
        print(f"🔹 어원: {origin}")
    if allomorph:
        print(f"🔹 이형태: {allomorph}")
    print()
    
    # 원어 정보
    original_language_infos = word_info.findall('original_language_info')
    if original_language_infos:
        print(f"🌍 원어 정보 ({len(original_language_infos)}개):")
        for idx, orig_info in enumerate(original_language_infos, start=1):
            original_language = orig_info.findtext('original_language')
            language_type = orig_info.findtext('language_type')
            print(f" {idx}. {original_language} ({language_type})")
        print()
    
    # 발음 정보
    pronunciation_infos = word_info.findall('pronunciation_info')
    if pronunciation_infos:
        print(f"🔊 발음 정보 ({len(pronunciation_infos)}개):")
        for idx, pron_info in enumerate(pronunciation_infos, start=1):
            pronunciation = pron_info.findtext('pronunciation')
            link = pron_info.findtext('link')
            print(f" {idx}. {pronunciation}")
            if link:
                print(f"    음성파일: {link}")
        print()
    
    # 활용/준말 정보
    conju_infos = word_info.findall('conju_info')
    if conju_infos:
        print(f"🔄 활용/준말 정보 ({len(conju_infos)}개):")
        for idx, conju_info in enumerate(conju_infos, start=1):
            print(f" {idx}.")
            
            # 활용 정보
            conjugation_infos = conju_info.findall('conjugation_info')
            if conjugation_infos:
                print(f"   📝 활용:")
                for conj_info in conjugation_infos:
                    conjugation = conj_info.findtext('conjugation')
                    print(f"     - {conjugation}")
                    
                    # 활용의 발음
                    conj_pron_infos = conj_info.findall('pronunciation_info')
                    for pron_info in conj_pron_infos:
                        pron = pron_info.findtext('pronunciation')
                        print(f"       🔊 [{pron}]")
            
            # 준말 정보
            abbreviation_infos = conju_info.findall('abbreviation_info')
            if abbreviation_infos:
                print(f"   ✂️ 준말:")
                for abbr_info in abbreviation_infos:
                    abbreviation = abbr_info.findtext('abbreviation')
                    print(f"     - {abbreviation}")
                    
                    # 준말의 발음
                    abbr_pron_infos = abbr_info.findall('pronunciation_info')
                    for pron_info in abbr_pron_infos:
                        pron = pron_info.findtext('pronunciation')
                        print(f"       🔊 [{pron}]")
        print()
    
    print("=" * 50)
    print("📚 의미 정보 (SENSE INFO)")
    print("=" * 50)
    
    # 기본 의미 정보
    pos = sense_info.findtext('pos')
    definition = sense_info.findtext('definition')
    word_type = sense_info.findtext('type')
    
    print(f"🔹 품사: {pos}")
    print(f"🔹 뜻풀이: {definition}")
    print(f"🔹 범주: {word_type}")
    print()
    
    # 학명
    scientific_name = sense_info.findtext('scientific_name')
    if scientific_name:
        print(f"🔬 학명: {scientific_name}")
        print()
    
    # 문형 정보
    pattern_infos = sense_info.findall('pattern_info')
    if pattern_infos:
        print(f"📝 문형 ({len(pattern_infos)}개):")
        for idx, pattern_info in enumerate(pattern_infos, start=1):
            pattern = pattern_info.findtext('pattern')
            print(f" {idx}. {pattern}")
        print()
    
    # 문법 정보
    grammar_infos = sense_info.findall('grammar_info')
    if grammar_infos:
        print(f"📚 문법 ({len(grammar_infos)}개):")
        for idx, grammar_info in enumerate(grammar_infos, start=1):
            grammar = grammar_info.findtext('grammar')
            print(f" {idx}. {grammar}")
        print()
    
    # 전문 분야
    cat_infos = sense_info.findall('cat_info')
    if cat_infos:
        print(f"🏷️ 전문 분야 ({len(cat_infos)}개):")
        for idx, cat_info in enumerate(cat_infos, start=1):
            cat = cat_info.findtext('cat')
            print(f" {idx}. {cat}")
        print()
    
    # 방언 지역
    region_infos = sense_info.findall('region_info')
    if region_infos:
        print(f"🗺️ 방언 지역 ({len(region_infos)}개):")
        for idx, region_info in enumerate(region_infos, start=1):
            region = region_info.findtext('region')
            print(f" {idx}. {region}")
        print()
    
    # 용례
    example_infos = sense_info.findall('example_info')
    if example_infos:
        print(f"💡 용례 ({len(example_infos)}개):")
        for idx, example_info in enumerate(example_infos, start=1):
            example = example_info.findtext('example')
            source = example_info.findtext('source')
            origin = example_info.findtext('origin')
            translation = example_info.findtext('translation')
            region = example_info.findtext('region')
            
            print(f" {idx}. {example}")
            if source: print(f"    출처: {source}")
            if origin: print(f"    원문: {origin}")
            if translation: print(f"    번역: {translation}")
            if region: print(f"    방언지역: {region}")
        print()
    
    # 관련 어휘
    relation_infos = sense_info.findall('relation_info')
    if relation_infos:
        print(f"🔗 관련 어휘 ({len(relation_infos)}개):")
        for idx, relation_info in enumerate(relation_infos, start=1):
            word_rel = relation_info.findtext('word')
            type_rel = relation_info.findtext('type')
            link_target_code = relation_info.findtext('link_target_code')
            link = relation_info.findtext('link')
            
            print(f" {idx}. {word_rel} ({type_rel})")
            if link: print(f"    링크: {link}")
        print()
    
    # 대역어
    translation_infos = sense_info.findall('translation_info')
    if translation_infos:
        print(f"🌐 대역어 ({len(translation_infos)}개):")
        for idx, translation_info in enumerate(translation_infos, start=1):
            translation = translation_info.findtext('translation')
            language_type = translation_info.findtext('language_type')
            print(f" {idx}. {translation} ({language_type})")
        print()
    
    # 역사 정보
    history_infos = sense_info.findall('history_info')
    if history_infos:
        print(f"📜 역사 정보 ({len(history_infos)}개):")
        for idx, history_info in enumerate(history_infos, start=1):
            word_form = history_info.findtext('word_form')
            desc = history_info.findtext('desc')
            allomorph = history_info.findtext('allomorph')
            remark = history_info.findtext('remark')
            
            print(f" {idx}.")
            if word_form: print(f"    어형변화/시기: {word_form}")
            if desc: print(f"    설명: {desc}")
            if allomorph: print(f"    이형태/이표기: {allomorph}")
            if remark: print(f"    관련정보: {remark}")
        print()
    
    # 멀티미디어
    multimedia_infos = sense_info.findall('multimedia_info')
    if multimedia_infos:
        print(f"🎥 멀티미디어 ({len(multimedia_infos)}개):")
        for idx, multimedia_info in enumerate(multimedia_infos, start=1):
            label = multimedia_info.findtext('label')
            type_mm = multimedia_info.findtext('type')
            link = multimedia_info.findtext('link')
            
            print(f" {idx}. {label} ({type_mm})")
            if link: print(f"    링크: {link}")
        print()
    
    # 규범 정보
    norm_infos = sense_info.findall('norm_info')
    if norm_infos:
        print(f"📋 규범 정보 ({len(norm_infos)}개):")
        for idx, norm_info in enumerate(norm_infos, start=1):
            type_norm = norm_info.findtext('type')
            role = norm_info.findtext('role')
            desc = norm_info.findtext('desc')
            
            print(f" {idx}. {type_norm}")
            if role: print(f"    관련조항: {role}")
            if desc: print(f"    설명: {desc}")
        print()
    
    # 관용구/속담
    proverb_infos = sense_info.findall('proverb_info')
    if proverb_infos:
        print(f"💭 관용구/속담 ({len(proverb_infos)}개):")
        for idx, proverb_info in enumerate(proverb_infos, start=1):
            word_prov = proverb_info.findtext('word')
            definition = proverb_info.findtext('definition')
            type_prov = proverb_info.findtext('type')
            link = proverb_info.findtext('link')
            
            print(f" {idx}. {word_prov} ({type_prov})")
            print(f"    뜻: {definition}")
            if link: print(f"    링크: {link}")
        print()
    
    # 수어 정보 링크
    sl_info_link = sense_info.findtext('sl_info_link')
    if sl_info_link:
        print(f"🤟 수어 정보: {sl_info_link}")

📖 단어 정보 (WORD INFO)
🔹 표제어: 가을
🔹 구성 단위: 어휘
🔹 고유어 여부: 고유어

🔊 발음 정보 (1개):
 1. 가을
    음성파일: http://media.korean.go.kr/front/search/searchResultView.do?file_no=20336

📚 의미 정보 (SENSE INFO)
🔹 품사: 명사
🔹 뜻풀이: 한 해의 네 철 가운데 셋째 철. 여름과 겨울의 사이이며, 달로는 9~11월, 절기(節氣)로는 입추부터 입동 전까지를 이른다.
🔹 범주: 일반어

💡 용례 (11개):
 1. 높디높은 {가을} 하늘.
 2. {가을이} 오다.
 3. {가을을} 맞다.
 4. {가을을} 보내다.
 5. 늦은 {가을이} 되면 서늘한 바람이 불고 나뭇잎이 떨어진다.
 6. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 7. 길 양옆으로 늘어선 벚나무들은 유난히도 {가을을} 타는지 어느덧 잎들을 다 떨어뜨리고 맨몸으로 서 있었다.
    출처: 조정래, 태백산맥
 8. 올해 {가을에는} 유난히 단풍이 곱게 들었다.
 9. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 10. 거개가 부자들이 주고 가난한 소작인들이 내가는데 봄에 쌀 한 섬이 {가을에} 쌀 두 섬이고, 봄에 보리 한 섬이 가을에 쌀로 한 섬입니다.
    출처: 송기숙, 녹두 장군
 11. 미스 헤이워드는 겨울 한 철만 빼면 초봄에서 {가을까지} 꽃밭 손질에 보통 정열을 쏟는 것이 아니었다. 휴일도 꽃밭 손질 때문에 기쁘다고 했다.
    출처: 박경리, 토지

🔗 관련 어휘 (41개):
 1. 갈002 (준말)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=449538
 2. 금추002 (비슷한말)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=104602
 3. 음중001 (비슷한말)
    링크: http:/

In [18]:
import requests
import xml.etree.ElementTree as ET

API_KEY = "561433A49188D3CB67FEDC2120EC2C87"
query = "가을"

url = "https://opendict.korean.go.kr/api/search"

params = {
    'key': API_KEY,
    'q': query,
    'req_type': 'xml',
    'num': 10,              # 여러 개 받아오고 첫 번째만 사용
    'advanced': 'y',

}

response = requests.get(url, params=params)
root = ET.fromstring(response.content)

# 첫 번째 <item> 가져오기
items = list(root.iter('item'))

if not items:
    print("검색 결과가 없습니다.")
else:
    item = items[0]
    word = item.findtext('word')

    print(f"🔹 단어: {word}")
    print()

    senses = item.findall('sense')
    for idx, sense in enumerate(senses, start=1):
        definition = sense.findtext('definition')
        pos = sense.findtext('pos')
        total = sense.findtext('total')
        type = sense.findtext('type')
        origin = sense.findtext('origin')


        print(f"🟢 의미 {idx}")
        print(f" - 품사: {pos}")
        print(f" - 뜻풀이: {definition}")
        print(f" - 범주: {type}")


🔹 단어: 가을

🟢 의미 1
 - 품사: 명사
 - 뜻풀이: 한 해의 네 철 가운데 셋째 철. 여름과 겨울의 사이이며, 달로는 9~11월, 절기(節氣)로는 입추부터 입동 전까지를 이른다.
 - 범주: 일반어


In [21]:
import requests
import xml.etree.ElementTree as ET

API_KEY = "561433A49188D3CB67FEDC2120EC2C87"
query = "가을"
url = "https://opendict.korean.go.kr/api/search"

params = {
    'key': API_KEY,
    'q': query,
    'req_type': 'xml',
    'num': 10,
    'part': 'exam',  # 용례 검색
}

response = requests.get(url, params=params)
root = ET.fromstring(response.content)

items = list(root.iter('item'))
if not items:
    print("검색 결과가 없습니다.")
else:
    for idx, item in enumerate(items[:5], start=1):  # 상위 5개만
        word = item.findtext('word')
        example = item.findtext('example')
        
        print(f"🟢 용례 {idx}")
        print(f" - 단어: {word}")
        print(f" - 용례: {example}")
        print()

🟢 용례 1
 - 단어: ㄹ
 - 용례: 가을이면 친구들과 단풍놀일 갔다.

🟢 용례 2
 - 단어: 가계^수익
 - 용례: 전남도 내의 올 가을무와 배추 등 김장 채소는 작황이 좋은 데다 높은 가격에 밭떼기 거래가 활발하게 이뤄지고 있어 재배 농민의 가계 수익에 큰 보탬을 주고 있다.

🟢 용례 3
 - 단어: 가권
 - 용례: 도내에 새로 제수된 수령은 충청도와 경상도의 예에 의하여, 가을이 되기까지는 가권을 데리고 부임하지 말게 하소서.

🟢 용례 4
 - 단어: 가당 식품
 - 용례: 어린이들의 주요 가당 식품의 1일 섭취 빈도는 여름에는 평균 4.1회, 가을에는 평균 3.1회였으며, 주요 가당 식품 중 특히 유제품류, 음료류, 과자류의 섭취 빈도가 높았고, 연령의 증가에 따라 주요 가당 식품의 1일 섭취 빈도는 증가하여 최대 1일 28회까지 나타났다.

🟢 용례 5
 - 단어: 가마득-히
 - 용례: 가을 들판에는 가마득히 높은 하늘이 펼쳐져 있었다.



In [27]:
import requests
import xml.etree.ElementTree as ET

API_KEY = "561433A49188D3CB67FEDC2120EC2C87"
query = "가을"

# 1단계: search API로 target_code 얻기
search_url = "https://opendict.korean.go.kr/api/search"
search_params = {
    'key': API_KEY,
    'q': query,
    'req_type': 'xml',
    'num': 10,
}

response = requests.get(search_url, params=search_params)
root = ET.fromstring(response.content)

item = root.find('.//item')
if item is None:
    print("검색 결과가 없습니다.")
else:
    word = item.findtext('word')
    target_code = item.find('sense').findtext('target_code')
    
    # 2단계: view API로 상세 정보 (용례 포함) 조회
    view_url = "https://opendict.korean.go.kr/api/view"
    view_params = {
        'key': API_KEY,
        'method': 'target_code',
        'q': target_code,
        'req_type': 'xml',
    }
    
    view_response = requests.get(view_url, params=view_params)
    view_root = ET.fromstring(view_response.content)
    
    # 결과 출력
    word_info = view_root.find('.//wordInfo')
    sense_info = view_root.find('.//senseInfo')
    
    word = word_info.findtext('word')
    pos = sense_info.findtext('pos')
    definition = sense_info.findtext('definition')
    word_type = sense_info.findtext('type')
    
    print(f"🔹 단어: {word}")
    print(f" - 품사: {pos}")
    print(f" - 뜻풀이: {definition}")
    print(f" - 범주: {word_type}")
    print()
    
    # 용례 추출
    example_infos = sense_info.findall('example_info')
    if example_infos:
        print(f"🟢 용례 ({len(example_infos)}개):")
        for idx, ex_info in enumerate(example_infos[:10], start=1):  # 최대 10개만
            example = ex_info.findtext('example')
            print(f" {idx}. {example}")

🔹 단어: 가을
 - 품사: 명사
 - 뜻풀이: 한 해의 네 철 가운데 셋째 철. 여름과 겨울의 사이이며, 달로는 9~11월, 절기(節氣)로는 입추부터 입동 전까지를 이른다.
 - 범주: 일반어

🟢 용례 (11개):
 1. 높디높은 {가을} 하늘.
 2. {가을이} 오다.
 3. {가을을} 맞다.
 4. {가을을} 보내다.
 5. 늦은 {가을이} 되면 서늘한 바람이 불고 나뭇잎이 떨어진다.
 6. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 7. 길 양옆으로 늘어선 벚나무들은 유난히도 {가을을} 타는지 어느덧 잎들을 다 떨어뜨리고 맨몸으로 서 있었다.
 8. 올해 {가을에는} 유난히 단풍이 곱게 들었다.
 9. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 10. 거개가 부자들이 주고 가난한 소작인들이 내가는데 봄에 쌀 한 섬이 {가을에} 쌀 두 섬이고, 봄에 보리 한 섬이 가을에 쌀로 한 섬입니다.


In [28]:
import requests
import xml.etree.ElementTree as ET

API_KEY = "561433A49188D3CB67FEDC2120EC2C87"
query = "가을"

# 1단계: search API로 target_code 얻기
search_url = "https://opendict.korean.go.kr/api/search"
search_params = {
    'key': API_KEY,
    'q': query,
    'req_type': 'xml',
    'num': 10,
}

response = requests.get(search_url, params=search_params)
root = ET.fromstring(response.content)

item = root.find('.//item')
if item is None:
    print("검색 결과가 없습니다.")
else:
    word = item.findtext('word')
    target_code = item.find('sense').findtext('target_code')
    
    # 2단계: view API로 상세 정보 조회
    view_url = "https://opendict.korean.go.kr/api/view"
    view_params = {
        'key': API_KEY,
        'method': 'target_code',
        'q': target_code,
        'req_type': 'xml',
    }
    
    view_response = requests.get(view_url, params=view_params)
    view_root = ET.fromstring(view_response.content)
    
    # 기본 정보
    word_info = view_root.find('.//wordInfo')
    sense_info = view_root.find('.//senseInfo')
    
    word = word_info.findtext('word')
    pos = sense_info.findtext('pos')
    definition = sense_info.findtext('definition')
    word_type = sense_info.findtext('type')
    
    print(f"🔹 단어: {word}")
    print(f" - 품사: {pos}")
    print(f" - 뜻풀이: {definition}")
    print(f" - 범주: {word_type}")
    print()
    
    # 학명
    scientific_name = sense_info.findtext('scientific_name')
    if scientific_name:
        print(f"🔬 학명: {scientific_name}")
        print()
    
    # 문형 정보
    pattern_infos = sense_info.findall('pattern_info')
    if pattern_infos:
        print(f"📝 문형 ({len(pattern_infos)}개):")
        for idx, pattern_info in enumerate(pattern_infos, start=1):
            pattern = pattern_info.findtext('pattern')
            print(f" {idx}. {pattern}")
        print()
    
    # 문법 정보
    grammar_infos = sense_info.findall('grammar_info')
    if grammar_infos:
        print(f"📚 문법 ({len(grammar_infos)}개):")
        for idx, grammar_info in enumerate(grammar_infos, start=1):
            grammar = grammar_info.findtext('grammar')
            print(f" {idx}. {grammar}")
        print()
    
    # 전문 분야
    cat_infos = sense_info.findall('cat_info')
    if cat_infos:
        print(f"🏷️ 전문 분야 ({len(cat_infos)}개):")
        for idx, cat_info in enumerate(cat_infos, start=1):
            cat = cat_info.findtext('cat')
            print(f" {idx}. {cat}")
        print()
    
    # 방언 지역
    region_infos = sense_info.findall('region_info')
    if region_infos:
        print(f"🗺️ 방언 지역 ({len(region_infos)}개):")
        for idx, region_info in enumerate(region_infos, start=1):
            region = region_info.findtext('region')
            print(f" {idx}. {region}")
        print()
    
    # 용례
    example_infos = sense_info.findall('example_info')
    if example_infos:
        print(f"💡 용례 ({len(example_infos)}개):")
        for idx, example_info in enumerate(example_infos, start=1):
            example = example_info.findtext('example')
            source = example_info.findtext('source')
            origin = example_info.findtext('origin')
            translation = example_info.findtext('translation')
            region = example_info.findtext('region')
            
            print(f" {idx}. {example}")
            if source: print(f"    출처: {source}")
            if origin: print(f"    원문: {origin}")
            if translation: print(f"    번역: {translation}")
            if region: print(f"    방언지역: {region}")
        print()
    
    # 관련 어휘
    relation_infos = sense_info.findall('relation_info')
    if relation_infos:
        print(f"🔗 관련 어휘 ({len(relation_infos)}개):")
        for idx, relation_info in enumerate(relation_infos, start=1):
            word_rel = relation_info.findtext('word')
            type_rel = relation_info.findtext('type')
            link_target_code = relation_info.findtext('link_target_code')
            link = relation_info.findtext('link')
            
            print(f" {idx}. {word_rel} ({type_rel})")
            if link: print(f"    링크: {link}")
        print()
    
    # 대역어
    translation_infos = sense_info.findall('translation_info')
    if translation_infos:
        print(f"🌐 대역어 ({len(translation_infos)}개):")
        for idx, translation_info in enumerate(translation_infos, start=1):
            translation = translation_info.findtext('translation')
            language_type = translation_info.findtext('language_type')
            print(f" {idx}. {translation} ({language_type})")
        print()
    
    # 역사 정보
    history_infos = sense_info.findall('history_info')
    if history_infos:
        print(f"📜 역사 정보 ({len(history_infos)}개):")
        for idx, history_info in enumerate(history_infos, start=1):
            word_form = history_info.findtext('word_form')
            desc = history_info.findtext('desc')
            allomorph = history_info.findtext('allomorph')
            remark = history_info.findtext('remark')
            
            print(f" {idx}.")
            if word_form: print(f"    어형변화/시기: {word_form}")
            if desc: print(f"    설명: {desc}")
            if allomorph: print(f"    이형태/이표기: {allomorph}")
            if remark: print(f"    관련정보: {remark}")
        print()
    
    # 멀티미디어
    multimedia_infos = sense_info.findall('multimedia_info')
    if multimedia_infos:
        print(f"🎥 멀티미디어 ({len(multimedia_infos)}개):")
        for idx, multimedia_info in enumerate(multimedia_infos, start=1):
            label = multimedia_info.findtext('label')
            type_mm = multimedia_info.findtext('type')
            link = multimedia_info.findtext('link')
            
            print(f" {idx}. {label} ({type_mm})")
            if link: print(f"    링크: {link}")
        print()
    
    # 규범 정보
    norm_infos = sense_info.findall('norm_info')
    if norm_infos:
        print(f"📋 규범 정보 ({len(norm_infos)}개):")
        for idx, norm_info in enumerate(norm_infos, start=1):
            type_norm = norm_info.findtext('type')
            role = norm_info.findtext('role')
            desc = norm_info.findtext('desc')
            
            print(f" {idx}. {type_norm}")
            if role: print(f"    관련조항: {role}")
            if desc: print(f"    설명: {desc}")
        print()
    
    # 관용구/속담
    proverb_infos = sense_info.findall('proverb_info')
    if proverb_infos:
        print(f"💭 관용구/속담 ({len(proverb_infos)}개):")
        for idx, proverb_info in enumerate(proverb_infos, start=1):
            word_prov = proverb_info.findtext('word')
            definition = proverb_info.findtext('definition')
            type_prov = proverb_info.findtext('type')
            link = proverb_info.findtext('link')
            
            print(f" {idx}. {word_prov} ({type_prov})")
            print(f"    뜻: {definition}")
            if link: print(f"    링크: {link}")
        print()
    
    # 수어 정보 링크
    sl_info_link = sense_info.findtext('sl_info_link')
    if sl_info_link:
        print(f"🤟 수어 정보: {sl_info_link}")

🔹 단어: 가을
 - 품사: 명사
 - 뜻풀이: 한 해의 네 철 가운데 셋째 철. 여름과 겨울의 사이이며, 달로는 9~11월, 절기(節氣)로는 입추부터 입동 전까지를 이른다.
 - 범주: 일반어

💡 용례 (11개):
 1. 높디높은 {가을} 하늘.
 2. {가을이} 오다.
 3. {가을을} 맞다.
 4. {가을을} 보내다.
 5. 늦은 {가을이} 되면 서늘한 바람이 불고 나뭇잎이 떨어진다.
 6. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 7. 길 양옆으로 늘어선 벚나무들은 유난히도 {가을을} 타는지 어느덧 잎들을 다 떨어뜨리고 맨몸으로 서 있었다.
    출처: 조정래, 태백산맥
 8. 올해 {가을에는} 유난히 단풍이 곱게 들었다.
 9. 어느새 여름이 가고 {가을이} 왔는지 서늘한 바람이 불었다.
 10. 거개가 부자들이 주고 가난한 소작인들이 내가는데 봄에 쌀 한 섬이 {가을에} 쌀 두 섬이고, 봄에 보리 한 섬이 가을에 쌀로 한 섬입니다.
    출처: 송기숙, 녹두 장군
 11. 미스 헤이워드는 겨울 한 철만 빼면 초봄에서 {가을까지} 꽃밭 손질에 보통 정열을 쏟는 것이 아니었다. 휴일도 꽃밭 손질 때문에 기쁘다고 했다.
    출처: 박경리, 토지

🔗 관련 어휘 (41개):
 1. 갈002 (준말)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=449538
 2. 금추002 (비슷한말)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=104602
 3. 음중001 (비슷한말)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=19990
 4. 겨울001 (참고 어휘)
    링크: http://opendict.korean.go.kr/dictionary/view?sense_no=450302
 5. 봄001 (참고 어휘)
    링크: http: